In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Analysis").getOrCreate()
print(spark.sparkContext.appName)

22/01/11 16:03:23 WARN Utils: Your hostname, Anishs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.90.85 instead (on interface en0)
22/01/11 16:03:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/11 16:03:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Analysis


In [2]:
import pandas as pd
import klib
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
file1 = "/Users/anishchapagain/Downloads/vericred_az_mercy_care_20211207.json"

In [4]:
df = spark.read.json(path=file1)
print("Count ",df.count())
print("Distinct Count ",df.distinct().count())

Count  113429


Distinct Count  113429


In [5]:
df.schema.names

['addresses',
 'group_affiliations',
 'hospital_affiliations',
 'networks',
 'provider',
 'specialties']

In [6]:
df.schema

StructType(List(StructField(addresses,ArrayType(StructType(List(StructField(address_string,StringType,true),StructField(city,StringType,true),StructField(languages,ArrayType(StructType(List(StructField(name,StringType,true),StructField(type,StringType,true))),true),true),StructField(office_name,StringType,true),StructField(phones,ArrayType(StructType(List(StructField(type,StringType,true),StructField(value,StringType,true))),true),true),StructField(state,StringType,true),StructField(street_line_1,StringType,true),StructField(street_line_2,StringType,true),StructField(zip,StringType,true))),true),true),StructField(group_affiliations,ArrayType(StructType(List(StructField(name,StringType,true))),true),true),StructField(hospital_affiliations,ArrayType(StringType,true),true),StructField(networks,ArrayType(StructType(List(StructField(name,StringType,true),StructField(tier,StringType,true))),true),true),StructField(provider,StructType(List(StructField(accepting_new_patients,BooleanType,true),

In [7]:
df.select('addresses').dtypes

[('addresses',
  'array<struct<address_string:string,city:string,languages:array<struct<name:string,type:string>>,office_name:string,phones:array<struct<type:string,value:string>>,state:string,street_line_1:string,street_line_2:string,zip:string>>')]

In [8]:
df.schema['addresses'].dataType

ArrayType(StructType(List(StructField(address_string,StringType,true),StructField(city,StringType,true),StructField(languages,ArrayType(StructType(List(StructField(name,StringType,true),StructField(type,StringType,true))),true),true),StructField(office_name,StringType,true),StructField(phones,ArrayType(StructType(List(StructField(type,StringType,true),StructField(value,StringType,true))),true),true),StructField(state,StringType,true),StructField(street_line_1,StringType,true),StructField(street_line_2,StringType,true),StructField(zip,StringType,true))),true)

In [9]:
df.select('addresses').printSchema()

root
 |-- addresses: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- address_string: string (nullable = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- languages: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- name: string (nullable = true)
 |    |    |    |    |-- type: string (nullable = true)
 |    |    |-- office_name: string (nullable = true)
 |    |    |-- phones: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- type: string (nullable = true)
 |    |    |    |    |-- value: string (nullable = true)
 |    |    |-- state: string (nullable = true)
 |    |    |-- street_line_1: string (nullable = true)
 |    |    |-- street_line_2: string (nullable = true)
 |    |    |-- zip: string (nullable = true)



In [10]:
df.select('provider').printSchema()

root
 |-- provider: struct (nullable = true)
 |    |-- accepting_new_patients: boolean (nullable = true)
 |    |-- facility_name: string (nullable = true)
 |    |-- facility_type: string (nullable = true)
 |    |-- first_name: string (nullable = true)
 |    |-- gender: string (nullable = true)
 |    |-- last_name: string (nullable = true)
 |    |-- license_number: string (nullable = true)
 |    |-- middle_name: string (nullable = true)
 |    |-- npi: string (nullable = true)
 |    |-- pcp: boolean (nullable = true)
 |    |-- pcp_id: string (nullable = true)
 |    |-- provider_type: string (nullable = true)
 |    |-- rating: struct (nullable = true)
 |    |    |-- scale: string (nullable = true)
 |    |    |-- score: string (nullable = true)
 |    |-- site_uid: string (nullable = true)
 |    |-- suffix: string (nullable = true)
 |    |-- title: string (nullable = true)
 |    |-- unparsed_name: string (nullable = true)



In [11]:
df.select('provider').schema

StructType(List(StructField(provider,StructType(List(StructField(accepting_new_patients,BooleanType,true),StructField(facility_name,StringType,true),StructField(facility_type,StringType,true),StructField(first_name,StringType,true),StructField(gender,StringType,true),StructField(last_name,StringType,true),StructField(license_number,StringType,true),StructField(middle_name,StringType,true),StructField(npi,StringType,true),StructField(pcp,BooleanType,true),StructField(pcp_id,StringType,true),StructField(provider_type,StringType,true),StructField(rating,StructType(List(StructField(scale,StringType,true),StructField(score,StringType,true))),true),StructField(site_uid,StringType,true),StructField(suffix,StringType,true),StructField(title,StringType,true),StructField(unparsed_name,StringType,true))),true)))

In [119]:
for colf in df.select('provider.*').schema:
    df.filter(col('provider'+colf)=='').select()
    break

TypeError: can only concatenate str (not "StructField") to str

In [23]:
facility=df.groupby("provider.facility_type").count().orderBy(col('count').desc())
facility.show()

+-------------+-----+
|facility_type|count|
+-------------+-----+
|         null|76525|
|             |36904|
+-------------+-----+



In [24]:
providerType = df.groupby("provider.provider_type").count().orderBy(col('count').desc())
providerType.show()

+-------------+-----+
|provider_type|count|
+-------------+-----+
|   individual|76525|
|     facility|36904|
+-------------+-----+



In [113]:
type(providerType)

pyspark.sql.dataframe.DataFrame

In [25]:
individual=providerType.filter(providerType.provider_type=="individual").select('count')

In [26]:
individual.show()

+-----+
|count|
+-----+
|76525|
+-----+



In [28]:
individual

DataFrame[count: bigint]

In [31]:
networks = df.select(explode(df.networks)).select('col.name','col.tier').groupBy(col("name")).count().orderBy(col('count').desc())
networks.show(truncate=False)

+-----------------------------------+-----+
|name                               |count|
+-----------------------------------+-----+
|Mercy Care Advantage               |20457|
|Mercy Care Long Term Care          |19895|
|Mercy Care Complete Care           |19798|
|Mercy Care Developmentally Disabled|19608|
|Mercy Care DCS CHP                 |14808|
|Mercy Care RBHA                    |12002|
|Behavioral Health Only             |6861 |
+-----------------------------------+-----+



In [ ]:
specs = df.select(explode(df.specialties)).select('col.name').groupBy(col("name")).count().orderBy(col('count').desc())
print('Total: {}',specs.count(),', Distinct: {}',specs.distinct().count())

In [64]:
specs.tail(20)

[Row(name='Dentist - Pediatric Dentist', count=4),
 Row(name='Physician Assistant - Psychiatry', count=4),
 Row(name='Surgical Services (Outpatient or ASC)', count=3),
 Row(name='Licensed Professional Counselor (LPC)', count=3),
 Row(name='Dental Hygienist', count=3),
 Row(name='Roentgenology (Diagnostic)', count=3),
 Row(name='Homeopathic', count=2),
 Row(name='Resid Trtm Ctr-Non-Secure (17Plusbeds)(Imd)', count=2),
 Row(name='Community or Rural Health Center', count=1),
 Row(name='THERAPIST-PHYSICAL', count=1),
 Row(name='Mhs Social Worker', count=1),
 Row(name='RN Midwife', count=1),
 Row(name='Dentist - Periodontist', count=1),
 Row(name='Dentist - Prosthodontist', count=1),
 Row(name='RN Adult Nurse Practitioner', count=1),
 Row(name='Dentist - Orthodontist', count=1),
 Row(name='RN Pediatric Nurse Practitioner', count=1),
 Row(name='Assisted Living Home (Formerly ACH)', count=1),
 Row(name='Pediatric - Neonatal/Perinatal Medicine', count=1),
 Row(name='BH Outpatient Clinic', coun

In [35]:

groups = df.select(explode(df.group_affiliations)).select('col.name').groupBy(col("name")).count().orderBy(col('count').desc())
print('Total: {}',groups.count(),', Distinct: {}',groups.distinct().count())
groups.show(truncate=False)

Total: {} 4730 , Distinct: {} 4730
+----------------------------------------------+-----+
|name                                          |count|
+----------------------------------------------+-----+
|Honorhealth Medical Group                     |2349 |
|Banner Hospital Based Physicians Arizona Llc  |1577 |
|Phoenix Childrens Medical Group               |1563 |
|Banner Physician Specialists Arizona Llc      |1563 |
|Sjhmc Physician Services                      |1392 |
|Banner Cancer Center Specialists Llc          |1044 |
|My Dr Now                                     |830  |
|District Medical Group Inc                    |816  |
|Obstetrix Medical Group Of Arizona Pc         |790  |
|El Rio Santa Cruz Neighborhood Health Center  |747  |
|Arizona Kidney Disease And Hypertension Center|618  |
|Mountain Park Health Center                   |589  |
|Ironwood Physicians Pc                        |511  |
|Md24 House Call                               |510  |
|Banner Physician Super Specia

In [38]:
df.select('hospital_affiliations').printSchema()

root
 |-- hospital_affiliations: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [105]:
info={}
for col in df.schema:
    print('Col: ',col.name, 'Type: ',col.dataType.typeName())
    if(col.dataType.typeName()=='array'):
#         print("\t",col.name,"---Null: ",col.dataType.containsNull,' :: ',col.nullable)
        if df.select(explode(col.name)).count()==0:
            info[col.name]='empty'
    else:
        

Col:  addresses Type:  array
	 addresses ---Null:  True  ::  True


Col:  group_affiliations Type:  array
	 group_affiliations ---Null:  True  ::  True
Col:  hospital_affiliations Type:  array
	 hospital_affiliations ---Null:  True  ::  True
Col:  networks Type:  array
	 networks ---Null:  True  ::  True
Col:  provider Type:  struct
Col:  specialties Type:  array
	 specialties ---Null:  True  ::  True


In [106]:
info

{'hospital_affiliations': 'empty'}

In [102]:
df.select(explode(df.hospital_affiliations)).count()

0

In [90]:
df.select('hospital_affiliations').tail(10)

[Row(hospital_affiliations=[]),
 Row(hospital_affiliations=[]),
 Row(hospital_affiliations=[]),
 Row(hospital_affiliations=[]),
 Row(hospital_affiliations=[]),
 Row(hospital_affiliations=[]),
 Row(hospital_affiliations=[]),
 Row(hospital_affiliations=[]),
 Row(hospital_affiliations=[]),
 Row(hospital_affiliations=[])]

In [108]:
hospitals = df.select(explode(df.hospital_affiliations)).select('col.name').groupBy(col("name")).count().orderBy(col('count').desc())

AnalysisException: Can't extract value from col#614: need struct type but got string

In [ ]:
print('Total: {}',hospitals.count(),', Distinct: {}',hospitals.distinct().count())
hospitals.show(truncate=False)

In [30]:
pdf = networks.show(networks.count(),truncate=False).toPandas()

+------------------------------------------+-----+
|name                                      |count|
+------------------------------------------+-----+
|Perform                                   |10220|
|Perform SE                                |10215|
|CentraChoice                              |10182|
|Open Access                               |10181|
|2020 Distinctions II                      |10169|
|2021 Distinctions II                      |10166|
|2020 Distinctions III                     |10162|
|2021 Distinctions III                     |10158|
|Robin Oak                                 |9974 |
|CentraChoice SE                           |8951 |
|HealthPartners Primary Clinic             |8824 |
|HealthPartners&reg; Freedom (Cost)        |8475 |
|Apex                                      |7331 |
|AchieveSM                                 |5360 |
|AchieveSM SE                              |5080 |
|TieredChoice                              |4364 |
|Atlas                         

In [110]:
facility.shape()

AttributeError: 'DataFrame' object has no attribute 'shape'

In [109]:
klib.cat_plot(facility)

AttributeError: 'DataFrame' object has no attribute 'shape'

In [12]:
spark.stop()